# Homework #8 Solutions
### Portfolio Theory and Risk Management I

## Imports

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
sns.set(rc={'figure.figsize':(15, 10)})

## Data

In [15]:
ltcm = pd.read_excel('../data/ltcm_exhibits_data_1.xlsx', sheet_name=1,skiprows=2).rename(columns={"Unnamed: 0": "Date"})\
                                                                             .set_index('Date').dropna()

ltcm.head()

,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance
Date,,,,
1994-03-01 00:00:00,1.1,-0.011,-0.013,0.99
1994-04-01 00:00:00,1.1,0.014,0.008,1.00
1994-05-01 00:00:00,1.2,0.068,0.053,1.05
1994-06-01 00:00:00,1.2,-0.039,-0.029,1.02
1994-07-01 00:00:00,1.4,0.116,0.084,1.10


In [16]:
SPY = pd.read_excel('../data/gmo_analysis_data_1.xlsx', sheet_name=2).set_index('Date')
rf = pd.read_excel('../data/gmo_analysis_data_1.xlsx', sheet_name=3).set_index('Date')

SPY['rf'] = rf['US3M']
SPY_ex = SPY.subtract(SPY['rf'], axis=0)['SPY'].to_frame('SPY ex')

SPY_ex.head()

,SPY ex
Date,
1993-02-28,0.0082
1993-03-31,0.0199
1993-04-30,-0.0281
1993-05-31,0.0244
1993-06-30,0.0011


In [17]:
ltcm.index = SPY_ex.loc['1994-03':'1998-07'].index
ltcm_ex = ltcm.subtract(SPY[['rf']], axis = 0).dropna().drop(columns = ['Index of Net Performance', 'rf'])
ltcm_ex['SPY (excess)'] = SPY_ex['SPY ex']

ltcm.head()

,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance
Date,,,,
1994-03-31,1.1,-0.011,-0.013,0.99
1994-04-30,1.1,0.014,0.008,1.00
1994-05-31,1.2,0.068,0.053,1.05
1994-06-30,1.2,-0.039,-0.029,1.02
1994-07-31,1.4,0.116,0.084,1.10


In [18]:
ltcm_ex.columns = ['Fund Capital ($billions)', 'Gross Performance (excess)', 'Net Performance (excess)', 'SPY (excess)']

ltcm_ex.head()

,Fund Capital ($billions),Gross Performance (excess),Net Performance (excess),SPY (excess)
Date,,,,
1993-02-28,NaN,NaN,NaN,0.0082
1993-03-31,NaN,NaN,NaN,0.0199
1993-04-30,NaN,NaN,NaN,-0.0281
1993-05-31,NaN,NaN,NaN,0.0244
1993-06-30,NaN,NaN,NaN,0.0011


# 1. LTCM

## 1.1 
### Describe LTCM’s investment strategy with regard to the following aspects:
- Securities traded
- Trading frequency
- Skewness (Do they seek many small wins or a few big hits?)
- Forecasting (What is behind their selection of trades?)

#### Solution: 
- LTCM traded across a wide range of securities. They are heavily involved in fixed income and credit, and they also have sizeable positions in equities. In all these asset classes, they trade a large number of securities, across global markets.

- LTCM's trading frequencies vary given that they house a wide range of strategies. Their largest bucket, the convergence trades, typically take weeks and months to converge. Overall, they are something of a medium-term frequency. Largely, they are not trying to arbitrage intraday movements, nor are they making long-term directional bets.

- LTCM largely is trying to pick up small premia in the convergence trades, which limits the upside of any individual trade, and leaves it substantial downside, given all the leverage. Of course, aggregating all their trades together can lead to positive skewness if they are properly diversified. But we would expect that their typical trade has a small positive mean with substantial negative skewness.

- For most their trades, LTCM is not making directional forecasts. Rather, they are trading spreads and pairs, forecasting relative convergence rather than absolute levels. In this sense, they do not rely on sophisticated forecasting models of the overall market but rather rely on models to forecast the size and speed of relative convergence in particular markets.

## 1.2
### What are LTCM’s biggest advantages over its competitors?

#### Solution:
Several advantages.
- Efficient financing. LTCM got very favorable terms on all financing—sometimes even zero haircut! Typically had small, if any, outlay.</li>
- Fund size. Have market power even in the large market of institutional wholesale.</li>
- Liquidity. LTCM has in place many mechanisms to ensure liquidity.</li>
- Long-term horizon. In financing and assessing trades, LTCM takes a relatively long-term view.
- Hedged. LTCM avoids taking too much default risk or explicit directional bets.

Then again, LTCM went bust in Aug. 1998, so maybe these advantages were not as strong as it
seemed!

## 1.3
### The case discusses four types of funding risk facing LTCM:
- collateral haircuts
- repo maturity
- equity redemption
- loan access
The case discusses specific ways in which LTCM manages each of these risks. Briefly discuss
them.

#### Solution:
The case discusses steps LTCM took to manage four types of funding risks.
- Collateral haircuts. For most trades, LTCM obtains 100% financing on a fully collateralized
basis. Furthermore, LTCM stress tests the haircuts across its asset classes.
- Repo. LTCM goes against the norm by entering into relatively long-maturity repo. While much of it is overnight, LTCM uses contracts that typically have maturity of 6-12 months. Furthermore, LTCM manages their aggregate repo maturity.
- Equity redemption. The firm is highly levered, so equity funding risk is especially important. LTCM restricts redemptions of equity year by year. The restriction is particularly strong in that unredeemed money is re-locked.
They also spread the redemption windows across the year to ensure there is never a possi- bility of immediate withdrawal of a large portion of equity.
- For debt funding, LTCM negotiated a revolving loan that has no Material Adverse Change clause. Thus, the availability of debt funding is not so highly correlated with fund perfor- mance.

## 1.4
### LTCM is largely in the business of selling liquidity and volatility. Describe how LTCM accounts for liquidity risk in their quantitative measurements.

#### Solution:
LTCM attempts to account for liquidity risk quantitatively by adjusting security correlations. For short-term horizons, LTCM assumes positive correlation between all trade cat- egories. Even if their net exposure to a strategy flips sides, they still assume positive correlation to the new net position.
Given the efforts of LTCM to hedge out obvious market risks, there are many strategies which would seem to have zero correlation. However, LTCM feels that liquidity concerns can cause the effective trading to be positively correlated.

## 1.5
### Is leverage risk currently a concern for LTCM?

#### Solution:
It would seem that leverage is not particularly dangerous at the moment. The fund’s volatility is relatively low, its VaR is relatively low, nor is it particularly high relative to the rest of the industry.
Moreover, the firm actively manages its funding risk which theoretically means it should be able to handle the natural risks of high leverage.
At the time of the case, the firm is trying to determine whether to further increase leverage. Subsequently, at the end of 1997 the fund returned about a third of its 7.5 billion equity capital to investors.
Of course, less than a year later, the fund blew up, but from the time of the case it’s hard to see the leverage risk.

## 1.6
### Many strategies of LTCM rely on converging spreads. LTCM feels that these are almost win/win situations because of the fact that if the spread converges, they make money. If it diverges, the trade becomes even more attractive, as convergence is still expected at a future date.

#### Solution:
What is the risk in these convergence trades?
Solution: About a year after the time of the case, the fund loses most of its value due to non-converging trades. So clearly there is some risk!
Positions are subject to liquidity risk. If market liquidity dries up or the markets become segmented, the divergent spreads can persist for a long time. This indeed happens later to LTCM. The trades that get them in trouble ultimately pay off, but not before LTCM blows up.
LTCM believes it can exit these convergence trades if they become too unprofitable. However, a stop-loss order is not the same as a put option. If the price jumps discontinuously through the stop-loss, then it is ineffective. Or a market may be paralyzed/illiquid when trying to execute the stop-loss. A put option does not need to worry about price impact, whereas a stop-loss does. Finally, a stop-loss ensures that an investor sells as soon as a security price hits a worst-case scenario, ensuring unfavorable market timing.


## 2 LTCM Risk Decomposition

## 2.1 Summary Stats

In [19]:
def summary_stats(df, annual_fac=12):
    stat_df = (df.mean()*annual_fac).to_frame('Mean')
    stat_df['Vol'] = df.std()*np.sqrt(annual_fac)
    stat_df['Sharpe'] = stat_df['Mean']/stat_df['Vol']
    
    return stat_df

def tail_risk_stats(df, annual_fac=12):
    tr_df = df.skew().to_frame('Skew')
    tr_df['Kurtosis'] = df.kurtosis()
    tr_df['VaR (0.05)'] = df.quantile(0.05)
    return tr_df

(a) For both the gross and net series of LTCM excess returns, report the mean, volatility, and
Sharpe ratios. (Annualize them.)

In [20]:
summary_stats(ltcm_ex[['Gross Performance (excess)','Net Performance (excess)','SPY (excess)']])

,Mean,Vol,Sharpe
Gross Performance (excess),NaN,NaN,NaN
Net Performance (excess),NaN,NaN,NaN
SPY (excess),0.0883,0.1461,0.6044


(b) Report the skewness, kurtosis, and (historic) VaR(.05).

In [21]:
tail_risk_stats(ltcm_ex[['Gross Performance (excess)','Net Performance (excess)','SPY (excess)']])

,Skew,Kurtosis,VaR (0.05)
Gross Performance (excess),NaN,NaN,NaN
Net Performance (excess),NaN,NaN,NaN
SPY (excess),-0.6025,1.2484,-0.0696


(c) Comment on how these stats compare to SPY and other assets we have seen. How much
do they differ between gross and net?

The summary stats are great for the data in the exhibits, and even better if cut off at the end of 1997, at the time of the case.

- Sharpe is very high relative to what we usually see.
- Volatility is sized similarly to SPY, and minimum return is not too bad.

Still, note that 
- SPY does amazing during this period, 
- so LTCM's returns while fantastic, are not an outlier relative to SPY.
- In fact, if we include the 1998 data through July, the tail-risk statistics look worse for LTCM than for SPY.

If we include the August crash,
- All the performance looks bad relative to SPY, particularly the tail-risk.

The difference between Gross and Net is not too large.
- Obviously, the net returns are lower, but given how high LTCM returns are (especially through) 1997, the net returns still deliver a fantastic Sharpe and overall profile. Though one can note they are a bit lower than SPY.

## 2.2 Market Regression

>$\tilde{r}^{LTCM}_{t} = \alpha + \beta^{m}\tilde{r}^{m}_{t}+\epsilon_{t}$

In [22]:
def regress(y, X, intercept = True, annual_fac=12):
    if intercept == True:
        X_ = sm.add_constant(X)
        reg = sm.OLS(y, X_).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc[r'$R^{2}$'] = reg.rsquared
        reg_df.loc['const'] *= annual_fac
    else:
        reg = sm.OLS(y, X).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc[r'$R^{2}$'] = reg.rsquared
    
    return reg_df

(a) Report $\alpha$ and $\beta^{m}$. Report the $R^{2}$ stat.

In [23]:
regress(ltcm_ex['Net Performance (excess)'], ltcm_ex['SPY (excess)']) 

,Regression Parameters
const,NaN
SPY (excess),NaN
$R^{2}$,NaN


(b) From this regression, does LTCM appear to be a “closet indexer”?

Definitely not. Correlation to the market is tiny and $\beta$ is small. 

(c) From the regression, does LTCM appear to deliver excess returns beyond the risk premium we expect from market exposure?

Yes, LTCM appears to deliver a large amount of excess returns beyond those expected from market exposure. The $\alpha$ of the regression is large at 13.15% annualized. 

## 2.3 Quadratic Regression

>$\tilde{r}^{LTCM}_{t} = \alpha + \beta_{1}\tilde{r}^{m}_{t}+ \beta_{2}(\tilde{r}^{m}_{t})^{2}+\epsilon_{t}$

(a) Report $\beta_{1}$, $\beta_{2}$, and the $R^{2}$ stat.

In [24]:
ltcm_ex['SPY (excess) squared'] = ltcm_ex['SPY (excess)'] **2

regress(ltcm_ex['Net Performance (excess)'], ltcm_ex[['SPY (excess)','SPY (excess) squared']]) 

,Regression Parameters
const,NaN
SPY (excess),NaN
SPY (excess) squared,NaN
$R^{2}$,NaN


(b) Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?

- The quadratic market factor does not add to $R^{2}$ to a significant degree.
- Thus, quadratic movements of SPY are not significant in explaining LTCM variation.

(c) From the regression evidence, does LTCM’s market exposure behave as if it is long market options or short market options?

LTCM appears to be short market options as it has negative exposure to market volatility (this is what the quadratic market factor represents). 

(d) Should we describe LTCM as being positively or negatively exposed to market volatility?

To the degree it has quadratic exposure, it is negative. This indicates LTCM underperforms particularly large SPY movements (whether on the upside or downside). Consequently, LTCM has negative exposure to market volatility. 

## 2.4 Asymmetric Regression

>$\tilde{r}^{LTCM}_{t} = \alpha + \beta\tilde{r}^{m}_{t}+ \beta_{u}max(\tilde{r}^{m}_{t}-k_{1}, 0)+ \beta_{d}max(k_{2}-\tilde{r}^{m}_{t}, 0)+\epsilon_{t}$

(a) Report $\beta$, $\beta_{u}$, $\beta_{d}$ and the $R^{2}$ stat.

In [25]:
k1 = .03
k2 = -.03

ltcm_ex['Up'] = (ltcm_ex['SPY (excess)'] - k1).clip(0)
ltcm_ex['Down'] = (k2 - ltcm_ex['SPY (excess)']).clip(0)

regress(ltcm_ex['Net Performance (excess)'], ltcm_ex[['SPY (excess)','Up','Down']])

,Regression Parameters
const,NaN
SPY (excess),NaN
Up,NaN
Down,NaN
$R^{2}$,NaN


(b) Is LTCM long or short the call-like factor? And the put-like factor?

- $\beta^{Up}$ is negative so LTCM appears short the call-like factor.
- $\beta^{Down}$ is positive so LTCM appears long the put-like factor.

(c) Which factor moves LTCM more, the call-like factor, or the put-like factor?

The magnitude of the Down market $\beta$ is larger, meaning that the put-like factor moves LTCM more.

(d) In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market’s upside? Short the market’s downside? Something else?

In the previous problem we commented that LTCM is negatively exposed to market vol. This volatility exposure must come more from being short the market's upside, as LTCM is short the call-like factor and long the put-like factor. 

## 3 The FX Carry Trade

## Two Data Sources

The original data set for the homework, `fx_carry_data.xlsx`, reported the **annualized** risk-free rate. 
- The Homework instructions did not tell you to divide the risk-free rates by 12, to change them to monthly rates. While I think this should have been apparent, it is fine if you used the data in its annualized form--but your results will be **very** different, and very unrealistic.

- If you did make this adjustment, you just divided the risk-free rates by 12. Note that LIBOR rates are reported using simple compounding, so we are fine to just multiply the rate by 1/12. This will be a bit different than the true, compounded, return. For our purposes it is not a big deal.

If you would like a clean set of data, use `fx_carry_data_v2.xlsx`.
- It reports the risk-free rates in a monthly scaling.
- It also pulls the LIBOR data from 1-month LIBOR instead of the original 3-month LIBOR. Thus, this set is more appropriate for our one-month trade-horizon considerations.

In [27]:
rf = pd.read_excel('../data/fx_carry_data_v2.xlsx', sheet_name = 1).set_index('DATE')

log_rf = np.log(1 + rf)

log_rf.head()

,USD1M,GBP1M,EUR1M,CHF1M,JPY1M
DATE,,,,,
1999-01-31,0.0041,0.0049,0.0026,0.0010,3.3458e-04
1999-02-28,0.0041,0.0046,0.0026,0.0010,2.3226e-04
1999-03-31,0.0041,0.0044,0.0025,0.0010,1.4270e-04
1999-04-30,0.0041,0.0044,0.0021,0.0008,9.8953e-05
1999-05-31,0.0041,0.0044,0.0021,0.0008,7.4997e-05


In [28]:
fx = pd.read_excel('../data/fx_carry_data_v2.xlsx', sheet_name = 2).set_index('DATE')

log_fx = np.log(fx)

log_fx.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,0.4982,0.1285,-0.3484,-4.7536
1999-02-28,0.4717,0.0949,-0.3712,-4.7766
1999-03-31,0.4787,0.0777,-0.3904,-4.7743
1999-04-30,0.4753,0.0549,-0.4225,-4.7827
1999-05-31,0.4713,0.0413,-0.4240,-4.7948


Timing of the risk-free rate:
- The data is defined such that the March value of the risk-free rate corresponds to the rate beginning in March and ending in April.
- In terms of the class notation, $r^{f,i}_{t,t+1}$ is reported at time $t$. (It is risk-free, so it is a rate from $t$ to $t+1$ but it is know at $t$.

## 3.1 The Static Carry Trade

In [29]:
log_rf_ex = log_rf.subtract(log_rf['USD1M'], axis=0)[log_rf.columns[1:]]

log_rf_ex_col = log_rf_ex.copy()

log_rf_ex_col.columns = log_fx.columns

ret = (log_rf_ex_col.shift() + log_fx.diff()).dropna()

ret.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-0.0257,-0.0351,-0.0259,-0.0268
1999-03-31,0.0075,-0.0187,-0.0222,-0.0016
1999-04-30,-0.0031,-0.0245,-0.0352,-0.0124
1999-05-31,-0.0037,-0.0155,-0.0048,-0.0160
1999-06-30,-0.0157,-0.0128,-0.0211,-0.0045


In [30]:
summary_stats(ret)

,Mean,Vol,Sharpe
USUK,-0.0035,0.0863,-0.0406
USEU,-0.0044,0.0947,-0.0459
USSZ,0.0043,0.0988,0.0437
USJP,-0.0174,0.0915,-0.1903


What differences do you see across currencies?

Mean returns are tiny and tend to be negative for currency pairs apart from the US dollar and the Swiss franc. The only returns of notable magnitude stem from the USD and JPY trade.

## 3.2 Implications for UIP

(a)  Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?

The results above are evidence against UIP, but not strong evidence.
- UIP implies that the mean excess return should be zero. The USD and JPY trade seems to have a mean excess return significantly different from zero.

- The evidence from the other currencies is not clear. Their mean excess return is not exactly zero, but they are so close to zero that they may not be statistically significant.

- Note that UIP makes no implication about the volatility of these excess returns.

(b) A long position in which foreign currency offered the best Sharpe ratio over the sample?
- Being long CHF would have delivered a small positive mean excess log return, but a poor Sharpe.
- The best trade would have been to be short JPY and long USD. This delivered a small but positive mean return and Sharpe.

(c) Are there any foreign currencies for which a long position earned a negative excess return (in USD) over the sample?

A long position in three of the currencies had negative mean excess (log) returns!

## 3.3 Predicting FX

> $s^{i}_{t+1}-s^{i}_{t}=\alpha^{i}+\beta^{i}(r^{f,$}_{t,t+1}-r^{f,i}_{t,t+1})+\epsilon^{i}_{t+1}$

(a) Make a table with columns corresponding to a different currency regression. Report the regression estimates $\alpha^{i}$ and $\beta^{i}$ in the first two rows. Report the $R^{2}$ stat in the third row.

In [31]:
ys = log_fx.diff().dropna()
Xs = -log_rf_ex.shift().dropna()

fx_reg_pred = pd.DataFrame(data = None, index = [r'$\alpha^{i}$',r'$\beta^{i}$',r'$R^{2}$'])

for i in range(0,len(ys.columns)):
    fx_reg_pred[ys.columns[i]] = regress(ys[ys.columns[i]],Xs[Xs.columns[i]])['Regression Parameters'].values
    
fx_reg_pred

,USUK,USEU,USSZ,USJP
$\alpha^{i}$,-0.0059,0.0070,0.0436,-0.0060
$\beta^{i}$,0.4858,-1.2564,-1.6466,0.3715
$R^{2}$,0.0004,0.0026,0.0039,0.0005


## 3.4 Dynamic Carry Trade

In [32]:
alphas = (fx_reg_pred.loc[r'$\alpha^{i}$'] / 12)
betas = fx_reg_pred.loc[r'$\beta^{i}$']
Xs.columns = fx_reg_pred.columns

ret_ex_forecast = alphas.values + ((betas - 1) * Xs)

ret_ex_forecast.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-6.8394e-05,-0.0028,-0.0046,-0.0029
1999-03-31,-2.3875e-04,-0.0029,-0.0046,-0.0029
1999-04-30,-3.2543e-04,-0.0031,-0.0046,-0.0030
1999-05-31,-3.2588e-04,-0.0038,-0.0050,-0.0030
1999-06-30,-3.3511e-04,-0.0039,-0.0050,-0.0030


(a) Use your regression estimates from Problem 3 along with the formula above to calculate the fraction of months for which the estimated FX risk premium positive. That is, for each $i$, calculate how often in the time-series we have:
> $E_{t}[\tilde{r}^{i}_{t+1}]>0$

In [33]:
month_frac = pd.DataFrame(data = None, columns = ret_ex_forecast.columns, index = ['% of Months'])

for col in ret_ex_forecast.columns:
    month_frac[col] = (len(ret_ex_forecast[ret_ex_forecast[col] > 0])/len(ret_ex_forecast)) * 100

ret_ex_forecast

month_frac

,USUK,USEU,USSZ,USJP
% of Months,23.8095,50.1832,63.0037,0.0


(b) Which currencies most consistently have a positive FX risk premium? And for which currencies does the FX risk premium most often go negative?

Though the static carry-trade of section 3.3 found that UK and EU have negative mean excess (log) returns, the dynamic carry trade finds that the time-varying forecast is often positive.

For JP, the forecast is negative throughout the sample due to the much higher interest rate in USD not being offset by appreciation of the Yen to the USD.

(c) Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.

Just as in HW7, we can use forecasts of excess returns to construct trading positions, (weights,) which vary with the forecasts. The resulting trading strategy should be better than the static carry trade.

That is, we could time the magnitude and direction of the currency trades instead of being passively 100\% invested in a currency according to the evidence in 3.3.

It is analagous to timing the trading of SPY according to the Earnings-Price Ratio instead of being passively 100\% invested in SPY.

Of course, it is an open question as to whether this will work given the weak R-squared in-sample, let alone out-of-sample.